In [1]:
# Env using Python 3.10.14
# pip install notebook==5.7.5

# the keras libraries
# pip install tensorflow Version: 2.17.0
import tensorflow as tf

from tensorflow.python.keras import models, layers

# Version: 3.4.1
from tensorflow import keras

# pip install keras-tuner
import keras_tuner as kt

In [2]:
# other libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# use for splitting the test and train data
from sklearn.model_selection import train_test_split

# for the creation of the cartesian product grid
from itertools import product

# for the timing of each test
import time
import datetime

In [3]:
import math
import json

# Basic conv net to achieve statistical significance

In [4]:
class Data_Handling:
    '''
    requirements:
        math
        numpy
        train_test_split from sklearn.model_selection
        
    '''
    def __init__(self, output_size=5, batch_size=128, ):
        self.__shape = 0
        self.__output_size = output_size
        self.__length = 0
        
        
    def load_data(self, label_path, data_path, split=0.2):
        # load the images and labels
        self.__labels = np.load(label_path)
        self.__data = np.load(data_path)
        self.__length = self.__labels.shape[0]
        self.__shape = self.__data[1].shape
        

        print("Loaded files of size:")
        print(f"Images: {self.__data.shape}\nLabels: {self.__labels.shape}")
        
    def split_data(self, split=0.2):
        # split and shuffle the data and labels
        self.__X_train, self.__X_test, self.__y_train, self.__y_test = train_test_split(
            self.__data, self.__labels, test_size=split)
        
    
#     def run_tuner(timer, tuner, max_epochs=50, batch_size=64, callbacks=[]):
#         SPLIT = 0.2
#         timer.start(test_name)

#         tuner.search(
#             self.__X_train, self.__y_train,
#             batch_size=batch_size,
#             epochs=max_epochs,
#             validation_split=SPLIT,
#             callbacks=callbacks,
#         )
#         # get and view the best performing hyper parameter set
#         best_hps = tuner.get_best_hyperparameters()[0]

#         test_duration = timer.stop(best_hps.values)
#         best_hps.values

    @property
    def shape(self, ):
        return self.__shape
    
    @property
    def output_size(self, ):
        return self.__output_size
    
    @property
    def X_train(self, ):
        return self.__X_train
    
    @property
    def y_train(self, ):
        return self.__y_train
    
    @property
    def X_test(self, ):
        return self.__X_test
    
    @property
    def y_test(self, ):
        return self.__y_test
    
    @property
    def length(self, ):
        return self.__length

In [7]:
dh = Data_Handling()

label_path = '../data/mitdb_labels_reduced.npy'
data_path = '../data/mitdb_data_reduced.npy'

dh.load_data(label_path=label_path, data_path=data_path)

Loaded files of size:
Images: (14190, 281, 362, 1)
Labels: (14190,)


In [8]:
dh.split_data(split=0.2)
print(dh.X_train.shape, dh.X_test.shape)

(11352, 281, 362, 1) (2838, 281, 362, 1)


In [10]:
# must prefix keras. unlike in the book
def build_basic_model(shape, output_size, callbacks):
    
    inputs = keras.Input(shape=shape)   
    x = keras.layers.Conv2D(filters=2, kernel_size=3, activation="relu")(inputs) 
    x = keras.layers.MaxPooling2D(pool_size=2)(x)
    x = keras.layers.Flatten()(x)
    
    outputs = keras.layers.Dense(output_size, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)

    # compile the model
    model.compile(optimizer="rmsprop",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    return model

In [13]:
# build the basic model and save the weights for use thorughout the testing 
model = build_basic_model(dh.shape, dh.output_size, callbacks=[])
model.save_weights('../weights/my_model_weights.h5')

In [89]:
# set the params
max_epochs = 50
vaidation_split = 0.2
batch_size = 256

# create the callbacks
metric = 'val_loss'
checkpoint_path = 'checkpoint_path.keras'

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor=metric, 
        patience=3
    )
]

In [90]:
# set seed to zero to be the same as that of the optimiser tests
# seed = 42
# tf.keras.utils.set_random_seed(seed) 
# build fresh model
basic_model = build_basic_model(dh.shape, dh.output_size, callbacks)
basic_model.load_weights('../weights/my_model_weights.h5')

# fit to the taining data
history = basic_model.fit(
    dh.X_train, 
    dh.y_train, 
    epochs=max_epochs, 
    validation_split=vaidation_split, 
    batch_size=batch_size, 
    callbacks=[callbacks]
)

Epoch 1/50
36/36 [==============================] - 16s 431ms/step - loss: 5469.6602 - accuracy: 0.3279 - val_loss: 3228.5994 - val_accuracy: 0.4963
Epoch 2/50
36/36 [==============================] - 14s 402ms/step - loss: 1304.4900 - accuracy: 0.5211 - val_loss: 250.0755 - val_accuracy: 0.7741
Epoch 3/50
36/36 [==============================] - 14s 388ms/step - loss: 304.1337 - accuracy: 0.7556 - val_loss: 71.6806 - val_accuracy: 0.8657
Epoch 4/50
36/36 [==============================] - 14s 401ms/step - loss: 99.4130 - accuracy: 0.8649 - val_loss: 53.4566 - val_accuracy: 0.8670
Epoch 5/50
36/36 [==============================] - 14s 399ms/step - loss: 23.3137 - accuracy: 0.9288 - val_loss: 19.5362 - val_accuracy: 0.9199
Epoch 6/50
36/36 [==============================] - 14s 392ms/step - loss: 6.2674 - accuracy: 0.9597 - val_loss: 9.5197 - val_accuracy: 0.9410
Epoch 7/50
36/36 [==============================] - 15s 419ms/step - loss: 1.8780 - accuracy: 0.9760 - val_loss: 7.5498 - va

In [92]:
# using the epoch as per the early stopping callback above
train_history = history.history[metric]
# get the index of the lowest recorded loss function (+ 1 to account for 0 idx)
best_epoch = get_best_epoch[metric](train_history) + 1

# build and train a fresh model for evaluation
basic_model = build_basic_model(dh.shape, dh.output_size, callbacks=[])
basic_model.load_weights('../weights/my_model_weights.h5')
# fit model on the entire training set by removing the validation_split param
basic_model.fit(
    dh.X_train, 
    dh.y_train, 
    epochs=best_epoch, 
    batch_size=batch_size, 
)

Epoch 1/11
45/45 [==============================] - 18s 381ms/step - loss: 4825.4570 - accuracy: 0.3307
Epoch 2/11
45/45 [==============================] - 17s 380ms/step - loss: 747.3987 - accuracy: 0.6426
Epoch 3/11
45/45 [==============================] - 17s 386ms/step - loss: 137.8913 - accuracy: 0.8294
Epoch 4/11
45/45 [==============================] - 17s 383ms/step - loss: 28.7769 - accuracy: 0.9227
Epoch 5/11
45/45 [==============================] - 17s 383ms/step - loss: 6.3042 - accuracy: 0.9583
Epoch 6/11
45/45 [==============================] - 16s 353ms/step - loss: 2.3356 - accuracy: 0.9691
Epoch 7/11
45/45 [==============================] - 17s 372ms/step - loss: 0.6828 - accuracy: 0.9845
Epoch 8/11
45/45 [==============================] - 17s 385ms/step - loss: 0.2793 - accuracy: 0.9897
Epoch 9/11
45/45 [==============================] - 17s 371ms/step - loss: 0.2441 - accuracy: 0.9928
Epoch 10/11
45/45 [==============================] - 17s 387ms/step - loss: 0.1077 

In [93]:
# evaluate performance on the unseen test data to see whether the basic model can beat 
# a the statistical significance calculated in the workbook [WORKBOOK]
basic_model.evaluate(dh.X_test, dh.y_test, batch_size=batch_size)

12/12 [==============================] - 2s 92ms/step - loss: 4.3531 - accuracy: 0.9475


[4.353099822998047, 0.9474982619285583]

In [23]:
pred = basic_model.predict(dh.X_test) 

89/89 [==============================] - 2s 18ms/step


In [84]:
get_best_epoch = {
    'accuracy': lambda x: np.argmax(x),
    'val_accuracy' : lambda x: np.argmax(x),
    'loss': lambda x: np.argmin(x),
    'val_loss' : lambda x: np.argmin(x) 
}

metric = 'val_accuracy'
get_best_epoch[metric](history.history[metric])   
    


5

In [27]:
p = np.argmax(pred, axis = 1)
r =dh.y_test
print(p,r)

[4 1 1 ... 4 1 2] [4 1 1 ... 4 1 2]


In [33]:
p = model.predict(dh.X_test)
p = np.argmax (p)
y_test=np.argmax(dh.y_test)

result = confusion_matrix(y_test, y_prediction , normalize='pred')
print(result)

89/89 [==============================] - 2s 17ms/step


NameError: name 'confusion_matrix' is not defined

In [ ]:
model.evaluate(X_test, y_test, batch_size=batch_size)

In [ ]:
model.history.history

## Grid search using keras library functinon

In [241]:
class HyperModel(kt.HyperModel):
    
    def __init__(self, num_classes, shape, filter_step=16, lr_step=0.1, weights=None):

            self.__num_classes = num_classes
            self.__shape = shape
            self.__filter_step = filter_step
            self.__lr_step = lr_step
            self.__weights = weights
            
    def build(self, hp):

        filters_1 = hp.Int(name="filters_1", min_value=16, max_value=32, step=self.__filter_step) 
        filters_2 = hp.Int(name="filters_2", min_value=32, max_value=64, step=self.__filter_step)
        # allows a zero setting 
        rate_1 = hp.Float(name="rate_1", min_value=0.2, max_value=0.5, step=self.__lr_step)
        
        inputs = keras.Input(shape=self.__shape)
        x = keras.layers.Conv2D(filters=filters_1, kernel_size=3, activation="relu")(inputs) 
        x = keras.layers.MaxPooling2D(pool_size=2)(x)
        x = keras.layers.Dropout(rate=rate_1)(x)
        x = keras.layers.Conv2D(filters=filters_2, kernel_size=3, activation="relu")(x) 
        x = keras.layers.MaxPooling2D(pool_size=2)(x)

        x = keras.layers.Flatten()(x)
        
        outputs = keras.layers.Dense(self.__num_classes, activation="softmax")(x)
        model = keras.Model(inputs=inputs, outputs=outputs)
        
        learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        
        model.compile(
            optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        
        # to ensure a level playing field between optimisers
        if self.__weights:
            model.load_weights('../weights/opt_test_weights.h5')
        
        return model

In [240]:
# get a set of initialized weights for the model
hp1 = kt.HyperParameters()
filter_step=8

hp1.Int(name="filters_1", min_value=16, max_value=32, step=filter_step) 
hp1.Int(name="filters_2", min_value=16, max_value=64, step=filter_step)
rate_1 = hp1.Float(name="rate_1", min_value=0, max_value=0.5, step=0.1)
hp1.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

hpm = HyperModel(num_classes=dh.output_size, shape=dh.shape, filter_step=16)
model = hpm.build(hp1)
weights_path = '../weights/opt_test_weights.h5'
model.save_weights(weights_path)

model.save('../models/untrained.keras')
# del model
# load_model('../models/untrained.keras')

In [111]:
class Tuner_Timer:
    
    def __init__(self):
        self.__start = None
        self.__end = None
        self.__duration = None
#         self.__results = {}
#         self.__tuners = []
#         self.__common_params = common_params
#         self.__callbacks = []
        
    def add_tuner(self, tuner):
        self.__tuners.append(tuner)
        
    def start(self):
        '''
        start the timer 
        '''
        self.__start = time.time()
        
    def stop(self):
        '''
        stop the timer and format the duration
        '''
        self.__end = time.time()
        self.__duration = self.__end - self.__start
        duration_string = time.strftime('%H:%M:%S', time.gmtime(self.__duration))

        return duration_string

In [112]:
def save_best_result(results_dict, results, tuner):

#     if tuner.project_name in results_dict.keys():
        
#         results_dict[tuner.project_name].update(results)
# #         print(results)
#     else:
#         results_dict[tuner.project_name] = results
        
    #save the current result to file
    f = open(f"{tuner.directory}/results.json", "w")

    json.dump(results, f, indent = 6)

    f.close()

In [179]:
def run_tuner(timer, tuner, batch_size, max_epochs, callbacks):
        '''
        run a given tuner saving the best parameter configuration to a timer object
        along with the total duration of the optimizer's run

        params:
            timer (Tuner_Timer) the timer used for recording results
            tuner (keras.Tuner) the current tuner under test 
            batch_size (int) batch for training 
            max_epochs (int) the maximum number of epochs to run if not stopped by early stopping
            callbacks (keras.callbacks) for early stopping
        '''
        # start timer
        print(f'Running optimizer {tuner.project_name}')
        timer.start()
        # the tuner will save results to the directory specified in the tuner constructor
        tuner.search(
            dh.X_train, dh.y_train,
            batch_size=batch_size,
            epochs=max_epochs,
            validation_split=0.2,
            callbacks=callbacks,
            verbose=1
        )

        # get the best performing hyper parameter set
        best_hps = tuner.get_best_hyperparameters()[0].values
        # stop timing and get the duration
        test_duration = timer.stop()
        
        return {'duration':test_duration, 'best_params':best_hps}

In [114]:
def retrain_and_evaluate(tuner):
    # get the object hp to rebuild a fresh model
    best_hps = tuners[0].get_best_hyperparameters()[0]
    # build a fresh model for retraining in order to find the point of overfitting
    model = hypermodel.build(best_hps)
    print('Finding best epoch')
    model.fit(dh.X_train[:1000], dh.y_train[:1000],
                batch_size=batch_size,
                epochs=max_epochs,
                validation_split=0.2,
                callbacks=callbacks,)

    # find best epoch since there seems no way to find this in the tuner
    best_epoch = np.argmin(model.history.history['val_accuracy'])
    # rebuild fresh model
    print(f'\nRetraining to best epoch: {best_epoch}')
    model = hypermodel.build(best_hps)
    # reterain on the entire set for the best epoch
    model.fit(dh.X_train, dh.y_train,
                batch_size=batch_size,
                epochs=best_epoch,)
    
    print('\nEvaluating model')
    loss, accuracy = model.evaluate(dh.X_test, dh.y_test)
    return loss, accuracy, best_epoch

In [253]:
hypermodel = HyperModel(num_classes=dh.output_size, shape=dh.shape, filter_step=16)
now = str(datetime.datetime.now())[5:10]

directory = f"start-{now}"
print(directory)
metrics = "val_loss"

common_params = {
#     'hypermodel': hypermodel, 
    'objective': metrics, 
    'executions_per_trial':1,
    'directory':directory,
    'tuner_id':'test_03',
    'overwrite':False,
}

callbacks = [
    keras.callbacks.EarlyStopping(monitor=metrics, patience=3),
]

start-08-28


In [254]:
batch_size = 128
max_epochs = 30

timer = Tuner_Timer()
best_results = {}
tuners = []

In [257]:
# RANDOM
tuners.append(kt.RandomSearch(build_optimizer_model, project_name='random', max_trials=40, **common_params))

# HYPERBAND
tuners.append(kt.Hyperband(build_optimizer_model, project_name='hyperband', factor=3, hyperband_iterations=1, **common_params))

# BAYES
tuners.append(kt.BayesianOptimization(build_optimizer_model, project_name='bayes', **common_params))

# GRID
tuners.append(kt.GridSearch(build_optimizer_model, project_name='grid', **common_params))

In [258]:
# run all optimizers
for seed, tuner in enumerate(tuners):
    # set seed for a level playing field between optimisers
#     tf.keras.utils.set_random_seed(seed)
    # run the optimizer
    best_result = run_tuner(timer, tuner, batch_size, max_epochs, callbacks)
    
#     # add best params to result dict and save current dict to file
#     save_best_result(best_results, best_result, tuner)
    if tuner.project_name in best_results.keys():
        
        best_results[tuner.project_name].update(best_result)
#         print(results)
    else:
        best_results[tuner.project_name] = best_result
    
    loss, accuracy, best_epoch = retrain_and_evaluate(tuners[0])
    
    best_results[tuner.project_name]['loss'] = loss
    best_results[tuner.project_name]['accuracy'] = accuracy
    #convert from int64 to make serializable
    best_results[tuner.project_name]['best_epoch'] = int(best_epoch)
    
     #save the current result to file
    f = open(f"{tuner.directory}/results.json", "w")

    json.dump(best_results, f, indent = 6)

    f.close()
    
    print(f'Loss: {loss}\tAccuracy: {accuracy}')
    

Trial 27 Complete [00h 36m 00s]
val_loss: 0.15532536804676056

Best val_loss So Far: 0.15532536804676056
Total elapsed time: 13h 23m 27s
Finding best epoch
Epoch 1/30
7/7 [==============================] - 14s 2s/step - loss: 1911.9928 - accuracy: 0.2725 - val_loss: 147.0729 - val_accuracy: 0.5250
Epoch 2/30
7/7 [==============================] - 12s 2s/step - loss: 787.3993 - accuracy: 0.3200 - val_loss: 274.2293 - val_accuracy: 0.2200
Epoch 3/30
7/7 [==============================] - 12s 2s/step - loss: 566.2479 - accuracy: 0.3525 - val_loss: 280.5002 - val_accuracy: 0.5250
Epoch 4/30
7/7 [==============================] - 13s 2s/step - loss: 726.5581 - accuracy: 0.3063 - val_loss: 295.4773 - val_accuracy: 0.1250

Retraining to best epoch: 3
Epoch 1/3
89/89 [==============================] - 176s 2s/step - loss: 384.8629 - accuracy: 0.3375
Epoch 2/3
89/89 [==============================] - 167s 2s/step - loss: 63.5409 - accuracy: 0.4828
Epoch 3/3
89/89 [==============================

In [259]:
best_results

{'random': {'duration': '13:12:02',
  'best_params': {'filters_1': 24,
   'rate_1': 0.30000000000000004,
   'learning_rate': 0.0001,
   'filters_2': 32},
  'loss': 1.2720849514007568,
  'accuracy': 0.6578576564788818,
  'best_epoch': 3},
 'hyperband': {'duration': '02:28:52',
  'best_params': {'filters_1': 32,
   'rate_1': 0.2,
   'learning_rate': 0.001,
   'tuner/epochs': 2,
   'tuner/initial_epoch': 0,
   'tuner/bracket': 4,
   'tuner/round': 0},
  'loss': 0.8689003586769104,
  'accuracy': 0.7431289553642273,
  'best_epoch': 3},
 'bayes': {'duration': '04:25:49',
  'best_params': {'filters_1': 24, 'rate_1': 0.4, 'learning_rate': 0.0001},
  'loss': 0.20684757828712463,
  'accuracy': 0.9467934966087341,
  'best_epoch': 12},
 'grid': {'duration': '13:23:26',
  'best_params': {'filters_1': 16, 'rate_1': 0.2, 'learning_rate': 0.0001},
  'loss': 0.9508249163627625,
  'accuracy': 0.7847075462341309,
  'best_epoch': 3}}

In [ ]:
tuners[0].results_summary(1)

In [116]:

now = datetime.datetime.now()
str(now)[:10]

'2024-08-28'

In [245]:
def build_untrained_model():

        inputs = keras.Input(shape=dh.shape)
        x = keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs) 
        x = keras.layers.MaxPooling2D(pool_size=2)(x)
        x = keras.layers.Dropout(rate=0.1)(x)
        x = keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) 
        x = keras.layers.MaxPooling2D(pool_size=2)(x)

        x = keras.layers.Flatten()(x)
        
        outputs = keras.layers.Dense(dh.output_size, activation="softmax")(x)
        model = keras.Model(inputs=inputs, outputs=outputs)
   
        return model

# build a model and save its weights for use throughout
model = build_untrained_model()
model.save_weights('../weights/untrained.h5')

In [256]:
def build_optimizer_model(hp):
    
    model = build_untrained_model()
    model.load_weights('../weights/untrained.h5')
    
    model.layers[0].filters = hp.Int(name="filters_1", min_value=16, max_value=32, step=8)
    model.layers[2].rate = hp.Float(name="rate_1", min_value=0.2, max_value=0.5, step=0.1)
    model.layers[3].filters = hp.Int(name="filters_1", min_value=16, max_value=32, step=8)
    
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
            optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
    
    return model

In [251]:
# common_params = {
#     'objective': metrics, 
#     'executions_per_trial':1,
#     'directory':directory,
#     'tuner_id':'test_03',
#     'overwrite':False,
# }

# tuner= kt.RandomSearch(
#     build_opt_model,
#     project_name='test_random', 
#     max_trials=4, 
# )

# tuner.search(
#             dh.X_train, dh.y_train,
#             batch_size=batch_size,
#             epochs=max_epochs,
#             validation_split=0.2,
#             callbacks=callbacks,
#             verbose=1
#         )


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
24                |24                |filters_1
0.4               |0.4               |rate_1
0.01              |0.01              |learning_rate

Epoch 1/30
71/71 [==============================] - 235s 3s/step - loss: 1917.2975 - accuracy: 0.3762 - val_loss: 1.5073 - val_accuracy: 0.2162
Epoch 2/30
 4/71 [>.............................] - ETA: 4:07 - loss: 1.3196 - accuracy: 0.5410

KeyboardInterrupt: 

[cite]  
The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + logfactor(max_epochs) and rounding it up to the nearest integer.

## Redundant code

In [ ]:
d = {
     0:
      {'hyperband':
       {'r1':100,'r2':150},
      'bayes':
       {'r1':100,'r2':150}
      },
      1:
      {'hyperband':
       {'r1':100,'r2':150},
      'bayes':
       {'r1':100,'r2':150}
      }
     }
    
d[2] = {"hyperband":{}}
d[2]['hyperband'] = {'r1':999,'r2':999}
d

### Results
Error "BiasGrad requires tensor size <= int32 max" with batch 256  
https://stackoverflow.com/questions/60414562/how-to-solve-the-biasgrad-requires-tensor-size-int32-max-invalidargumenterr